<a href="https://colab.research.google.com/github/ColocadoJoyce/CCADMACL_EXERCISES/blob/main/Colocado_Exercise1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 1

Use all feature selection methods to find the best features

## Dataset Information

## Features

Number of Instances: 20640

Number of Attributes: 8 numeric, predictive attributes and the target

Attribute Information:

MedInc - median income in block group

HouseAge - median house age in block group

AveRooms - average number of rooms per household

AveBedrms - average number of bedrooms per household

Population - block group population

AveOccup - average number of household members

Latitude - block group latitude

Longitude - block group longitude

## Target
The target variable is the median house value for California districts, expressed in hundreds of thousands of dollars ($100,000).

In [1]:
from sklearn.datasets import fetch_california_housing

import pandas as pd

In [2]:
housing = fetch_california_housing(as_frame=True)
df = pd.concat([housing.data, housing.target], axis=1)

In [3]:
df_housing = housing
df_housing_features = pd.DataFrame(df_housing.data, columns = df_housing.feature_names)
df_housing_target = pd.DataFrame(df_housing.target, columns=['target'])


In [4]:
score_list = {}

In [5]:
X = housing.data
y = housing.target

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)

from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(X_train, y_train)

# Predict on the test data
y_pred = model.predict(X_test)

from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)

RMSE: 0.7219777996952079


1. Use any filter method to select the best features

In [7]:
#F_classif  Method
from sklearn.feature_selection import f_classif

threshold = 5 # the number of most relevant features
high_score_features = []
feature_scores = f_classif(housing.data, housing.target.values.ravel())[0]


In [9]:
X = df_housing_fc
y = df_housing_target

In [10]:

for score, f_name in sorted(zip(feature_scores, housing.data.columns), reverse=True)[:threshold]:
      # print(f_name, score)
      high_score_features.append(f_name)

house_fc = housing.data[high_score_features]
house_fc.columns

Index(['MedInc', 'Latitude', 'Population', 'HouseAge', 'Longitude', 'MedInc',
       'Latitude', 'Population', 'HouseAge', 'Longitude'],
      dtype='object')

In [11]:

X = house_fc
y = housing.target

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)


In [13]:

model = LinearRegression()

model.fit(X_train, y_train)

# Predict on the test data
y_pred = model.predict(X_test)

In [14]:
model.fit(X,y)

LinearRegression()

In [15]:
rmse_fc = np.sqrt(np.mean((y_test - y_pred) ** 2))

# Print the RMSE
print(f'Root Mean Squared Error (RMSE): {rmse}')

Root Mean Squared Error (RMSE): 0.7219777996952079


In [16]:
score_list["F-Classif"] = rmse

2. Use any wrapper method to select the best features

In [17]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE

threshold = 5
model_rf = RandomForestRegressor(n_estimators=500, random_state=0, max_depth = 3)
selector = RFE(model_rf, n_features_to_select=5, step=1)

selector = selector.fit(housing.data, housing.target.values.ravel())
selector_ind = selector.get_support()
df_housing_rfe = df_housing_features.iloc[:, selector_ind]
df_housing_rfe.columns

Index(['MedInc', 'HouseAge', 'AveRooms', 'AveOccup', 'Latitude'], dtype='object')

In [18]:
X = df_housing_rfe
y = housing.target



In [19]:
df_housing_rfe


,MedInc,HouseAge,AveRooms,AveOccup,Latitude
0,8.3252,41.0,6.984127,2.555556,37.88
1,8.3014,21.0,6.238137,2.109842,37.86
2,7.2574,52.0,8.288136,2.802260,37.85
3,5.6431,52.0,5.817352,2.547945,37.85
4,3.8462,52.0,6.281853,2.181467,37.85
...,...,...,...,...,...
20635,1.5603,25.0,5.045455,2.560606,39.48
20636,2.5568,18.0,6.114035,3.122807,39.49
20637,1.7000,17.0,5.205543,2.325635,39.43
20638,1.8672,18.0,5.329513,2.123209,39.43


In [20]:
df_housing_target

,target


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)

In [22]:


from sklearn.linear_model import LinearRegression
model = LinearRegression()

model.fit(X_train, y_train)

# Predict on the test data
y_pred = model.predict(X_test)

model.fit(X,y)

rmse_wrapper = np.sqrt(mean_squared_error(y_test, y_pred))
print("Wrapper RMSE:", rmse)

Wrapper RMSE: 0.7219777996952079


3. Use any embedded methood to select the best features

In [23]:
X = housing.data
y = housing.target

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)




In [26]:
from sklearn.feature_selection import SelectFromModel

model_rf = RandomForestRegressor(n_estimators=500, random_state=42, max_depth = 3)
model_rf.fit(X_train, y_train)

y_pred = model_rf.predict(X_test)

sel_sfm = SelectFromModel(model_rf, prefit=True)
sel_sfm_index = sel_sfm.get_support()
df_housing_sfm = df_housing_features.iloc[:, sel_sfm_index]
df_housing_sfm.columns


Index(['MedInc', 'AveOccup'], dtype='object')

In [27]:
rmse_embedded = np.sqrt(mean_squared_error(y_test, y_pred))
print("Embedded RMSE:", rmse)

Embedded RMSE: 0.7219777996952079


In [28]:
print("Default:", rmse)
print("Filter Method:", rmse_fc)
print("Wrapper Method:", rmse_wrapper)
print("Embedded Method:", rmse_embedded)

Default: 0.7219777996952079
Filter Method: 0.7390401074080456
Wrapper Method: 0.7985450982024661
Embedded Method: 0.7844874438510104
